In [3]:
import numpy as np
import random
from collections import defaultdict

# Ambiente de Controle de Glicose
class DiabetesEnv:
    def __init__(self):
        self.reset()

    def reset(self):
        # Redefine o ambiente para um nível de glicose e outras variáveis iniciais.
        self.glucose_level = 120  # Um valor inicial médio de glicose.
        self.insulin = 0
        self.activity_level = 0
        self.time_of_day = 0
        self.state = (self.glucose_level, self.insulin, self.activity_level, self.time_of_day)
        return self.state

    def step(self, action):
        # Atualiza o estado do ambiente com base na ação tomada pelo agente.
        if action == 'insulin_small':
            self.glucose_level -= random.uniform(10, 15)
        elif action == 'insulin_medium':
            self.glucose_level -= random.uniform(20, 25)
        elif action == 'insulin_large':
            self.glucose_level -= random.uniform(30, 35)
        elif action == 'exercise_low':
            self.glucose_level -= random.uniform(5, 10)
        elif action == 'exercise_medium':
            self.glucose_level -= random.uniform(10, 20)
        elif action == 'exercise_high':
            self.glucose_level -= random.uniform(20, 30)
        elif action == 'food_high_glycemic':
            self.glucose_level += random.uniform(30, 50)
        elif action == 'food_low_glycemic':
            self.glucose_level += random.uniform(10, 20)

        # Atualiza a hora do dia
        self.time_of_day = (self.time_of_day + 1) % 3
        self.state = (self.glucose_level, self.insulin, self.activity_level, self.time_of_day)

        # Função de recompensa melhorada
        if 70 <= self.glucose_level <= 140:
            reward = 10  # Alta recompensa por manter a glicose em nível saudável.
        else:
            reward = -10  # Penalidade para níveis de glicose fora do intervalo desejado.

        # Define se o episódio termina
        done = self.glucose_level < 40 or self.glucose_level > 400
        return self.state, reward, done


In [8]:
# Agente de Q-Learning
class QLearningAgent:
    def __init__(self, actions, learning_rate=0.1, discount_factor=0.9, exploration_rate=1.0, exploration_decay=0.995):
        self.q_table = defaultdict(lambda: np.zeros(len(actions)))
        self.actions = actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = exploration_rate
        self.epsilon_decay = exploration_decay

    def choose_action(self, state, explore=True):
        # Escolhe ação baseada em exploração/exploração ou apenas na política aprendida.
        if explore and random.uniform(0, 1) < self.epsilon:
            return random.choice(self.actions)
        else:
            return self.actions[np.argmax(self.q_table[state])]

    def learn(self, state, action, reward, next_state):
        action_idx = self.actions.index(action)
        q_predict = self.q_table[state][action_idx]
        q_target = reward + self.gamma * np.max(self.q_table[next_state])
        self.q_table[state][action_idx] += self.lr * (q_target - q_predict)
        # Decai a taxa de exploração
        self.epsilon = max(0.1, self.epsilon * self.epsilon_decay)


In [14]:
# Treinamento com Feedback
actions = [
    'insulin_small', 'insulin_medium', 'insulin_large',
    'exercise_low', 'exercise_medium', 'exercise_high',
    'food_high_glycemic', 'food_low_glycemic'
]

env = DiabetesEnv()
agent = QLearningAgent(actions)

num_episodes = 1000
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    episode_glucose_levels = []

    while not done:
        action = agent.choose_action(state, explore=True)
        next_state, reward, done = env.step(action)
        agent.learn(state, action, reward, next_state)

        state = next_state
        total_reward += reward
        episode_glucose_levels.append(env.glucose_level)

    # Exibe informações a cada 100 episódios para verificar progresso
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode + 1}, Total Reward: {total_reward}")
        print(f"Níveis de glicose no episódio: {episode_glucose_levels}")


Episode 100, Total Reward: 0
Níveis de glicose no episódio: [110.07175443671869, 95.32039343250885, 84.21655065076403, 77.61091692192934, 66.07940419348348, 54.37749465363839, 43.31453331407481, 28.990617603182727]
Episode 200, Total Reward: 0
Níveis de glicose no episódio: [114.2400342931151, 103.9192174034372, 93.10878497302589, 80.7762599037873, 68.20216028941906, 56.421551579847645, 46.3225088968341, 35.53839534704954]
Episode 300, Total Reward: 0
Níveis de glicose no episódio: [114.86633272494802, 103.21082984302444, 90.87664543140865, 80.87409935872054, 68.08880526072456, 53.5423574238565, 41.92870850845014, 27.11347639445674]
Episode 400, Total Reward: 0
Níveis de glicose no episódio: [110.59131810996104, 88.20872406457242, 76.95558658119329, 64.19793881610089, 49.27815404903092, 38.08862099420851]
Episode 500, Total Reward: 10
Níveis de glicose no episódio: [113.48604157637244, 102.27446568697204, 90.66220841700061, 77.67882124626234, 66.29071354192851, 51.302410338229976, 38.6

In [20]:
# Validação do Agente Treinado
print("\nIniciando Fase de Validação\n")
validation_episodes = 100  # Número de episódios de validação
validation_rewards = []
healthy_episodes_count = 0

for episode in range(validation_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    episode_glucose_levels = []
    episode_healthy = True  # Suponha que o episódio é saudável a princípio

    while not done:
        action = agent.choose_action(state, explore=False)  # Apenas política aprendida (sem exploração)
        next_state, reward, done = env.step(action)

        state = next_state
        total_reward += reward
        episode_glucose_levels.append(env.glucose_level)

        # Verifica se o nível de glicose saiu da faixa saudável
        if not (70 <= env.glucose_level <= 140):
            episode_healthy = False

    validation_rewards.append(total_reward)

    # Conta este episódio como saudável se os níveis de glicose permaneceram na faixa
    if episode_healthy:
        healthy_episodes_count += 1

    print(f"Validation Episode {episode + 1}, Total Reward: {total_reward}")
    print(f"Níveis de glicose no episódio: {episode_glucose_levels}")

# Análise dos Resultados de Validação
average_reward = np.mean(validation_rewards)
percentage_healthy_episodes = (healthy_episodes_count / validation_episodes) * 100

print(f"\nRecompensa média na fase de validação: {average_reward}")
print(f"Porcentagem de episódios com níveis de glicose saudáveis: {percentage_healthy_episodes:.2f}%")

# Avaliação final
if percentage_healthy_episodes > 70:
    print("Sim, o agente foi capaz de manter os níveis de glicose dentro do intervalo desejado na maioria dos episódios.")
else:
    print("Não, o agente não conseguiu manter os níveis de glicose no intervalo desejado na maioria dos episódios.")



Iniciando Fase de Validação

Validation Episode 1, Total Reward: 10
Níveis de glicose no episódio: [112.55880485498453, 99.47317177562189, 87.66272844987999, 74.15663109721629, 59.63768584434975, 45.082326655393885, 34.32299512199247]
Validation Episode 2, Total Reward: 0
Níveis de glicose no episódio: [114.65483670100664, 102.68383084109534, 92.0623672542888, 81.38798639584319, 67.84534882521423, 57.25708907568082, 47.0901400371705, 33.73938039125234]
Validation Episode 3, Total Reward: 0
Níveis de glicose no episódio: [114.5517638569993, 103.70940917314057, 91.75691255487516, 77.52407060584629, 65.48938249698114, 51.32779965300462, 40.55758102674842, 29.582144007515137]
Validation Episode 4, Total Reward: 0
Níveis de glicose no episódio: [113.40378819704733, 102.76912320598204, 90.04198722875125, 76.36777864948162, 66.15563180844194, 51.859091372075405, 41.14341776009332, 30.322968435321894]
Validation Episode 5, Total Reward: 10
Níveis de glicose no episódio: [114.06308515015978, 9